In [2]:
import logging
import os
import pdb
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import xarray
from matplotlib.colors import ListedColormap
from scipy.ndimage import maximum_filter

from contingency_table import accuracy, csi, combine_hmfn, ets, hk, hss, pod, pofd, total
from hwt2024 import Model

sns.set_theme()
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s", force=True)
tmpdir = Path(os.getenv("TMPDIR"))

In [8]:
# Discrete color scheme for hit, miss, false alarm, and null
cMap = ListedColormap(["tab:green", "blue", "orangered", "white"])

models = [
    Model(
        "fv3",
        nmem=10,
        v=["MXUPHL0_1km_max", "MXUPHL0_3km_max", "MXUPHL2_5km_max"],
        lead_time_days=8,
    ),
    Model(
        "mpas",
        nmem=5,
        v=["updraft_helicity_max01", "updraft_helicity_max03", "updraft_helicity_max"],
        lead_time_days=5,
    ),
]
lead_time_days = range(5)

In [ ]:
valid_date = pd.to_datetime("20240520")
filter_size = 5
thresh = 150
rpt_type = "hail"


rptfile = tmpdir / f"near_rpt.{valid_date}.nc"
logging.info(f"open existing {rptfile}")
near_rpts = xarray.open_dataarray(rptfile)
conus_mask = xarray.open_dataarray(
    Path(os.getenv("HOME").replace(os.getenv("USER"), "ahijevyc")) / "HWT_2024.conus.nc"
)

plt.close("all")
for dayForecast in range(1,6):
    ofile = tmpdir / f"hmfn.{valid_date.strftime('%Y%m%d')}.day{dayForecast}.{filter_size}.{thresh}.{rpt_type}.png"
    fig, axes = plt.subplots(ncols=len(models), figsize=(11, 5))
    for model, ax in zip(models, axes):
        ncfile = tmpdir / f"forecast_yes.{model}.{valid_date}.{lead_time_days.stop}.nc"
        logging.info(f"open {ncfile}")
        fy = xarray.open_dataarray(ncfile)
        
        logging.info(f"size-{filter_size} filter")
        # scipy.ndimage filter requires smoothing dimensions ("lon", "lat"; axes argument),
        # to be specified with their index positions. You can't refer to them by their string names.
        londim = fy.dims.index("lon")
        latdim = fy.dims.index("lat")
        smoothfy = maximum_filter(fy, size=filter_size, axes=(londim,latdim))
        fy = xarray.DataArray(smoothfy, dims=fy.dims, coords=fy.coords, attrs=fy.attrs)
        
        logging.info("hit")
        h = fy * near_rpts
        logging.info("miss")
        m = ~fy * near_rpts
        logging.info("fa")
        f = fy * ~near_rpts
        logging.info("null")
        n = ~fy * ~near_rpts

        contingency_table = combine_hmfn(h,m,f,n).sum(dim=["lon","lat"])
        sel = dict(thresh=thresh, variable=model.v[2], dayForecast=dayForecast, rpt_type=rpt_type)
        ax.text(0.1, 0.1, f"ets {ets(contingency_table.sel(sel)):.3f}", transform=ax.transAxes) 
        logging.info("hmfn")
        hmfn = m + f * 2 + n * 3
    
        logging.info("Keep value where conus_mask is True; otherwise set to nan.")
        hmfn = hmfn.where(conus_mask)
        logging.info("plot")
        qax = hmfn.sel(sel).plot(
            cmap=cMap, ax=ax, add_colorbar=False
        )
        # hide ticks. coordinates are not really "lon" and "lat" anyway. they are model indices.
        qax.axes.get_xaxis().set_ticks([])
        qax.axes.get_yaxis().set_ticks([])
        qax.axes.set_ylabel(None)
        # label xaxis with model
        qax.axes.set_xlabel(model)

    # Nice hmfn colorbar
    cbar = plt.colorbar(qax, ax=axes, orientation="horizontal", aspect=32)
    cbar.ax.get_xaxis().set_ticks([])
    # label in center of color
    for j, lab in enumerate(["hit", "miss", "false alarm", "correct null"]):
        cbar.ax.text(3 * (1 + (2 * j)) / 8.0, 0.5, lab, ha="center", va="center")

    # title
    valid_range = valid_date + pd.to_timedelta(12, unit="hour"), valid_date + pd.to_timedelta(36, unit="hour")
    vrstr = valid_range[0].strftime("%Y-%m-%d %Hz - ") + valid_range[1].strftime("%Y-%m-%d %Hz")
    plt.suptitle(vrstr + f" thresh={thresh} filter={filter_size} {rpt_type}")
    fig.savefig(ofile, dpi=150)
    print(ofile)

2024-06-27 14:50:51,671 open existing /glade/derecho/scratch/ahijevyc/tmp/near_rpt.2024-05-20 00:00:00.nc
2024-06-27 14:50:52,162 open /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.2024-05-20 00:00:00.5.nc
2024-06-27 14:50:52,222 size-5 filter
2024-06-27 14:52:27,196 hit
2024-06-27 14:52:37,767 miss
2024-06-27 14:52:48,450 fa
2024-06-27 14:52:57,601 null
2024-06-27 14:53:24,336 hmfn
2024-06-27 14:54:53,428 Keep value where conus_mask is True; otherwise set to nan.
2024-06-27 14:55:36,544 plot
2024-06-27 14:55:37,170 open /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.mpas.2024-05-20 00:00:00.5.nc
2024-06-27 14:55:37,230 size-5 filter
2024-06-27 14:55:53,350 hit
2024-06-27 14:55:56,249 miss
2024-06-27 14:56:00,755 fa
2024-06-27 14:56:03,609 null
2024-06-27 14:56:33,816 hmfn
2024-06-27 14:58:39,409 Keep value where conus_mask is True; otherwise set to nan.
2024-06-27 15:00:14,941 plot


/glade/derecho/scratch/ahijevyc/tmp/hmfn.20240520.day1.5.150.hail.png


2024-06-27 15:00:36,798 open /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.2024-05-20 00:00:00.5.nc
